In [3]:
import langchain
langchain.debug = True

from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage
from langchain.tools import StructuredTool

In [4]:
llm = ChatOpenAI(temperature=0)

system_message = SystemMessage(content='''Act as a helpful AI interview assistant, 
                               You will either be having a conversation to generate content or calling a function to help the user.
                               
                               You must follow the following principles:
                               - Always be asking the user a follow up question, especially after returning the results from a previous tool.
                               - Ask one question at a time.
                               - You must always be in interview mode (asking questions) or in tool mode (calling a function). 
                               - Ask the user what topic they want to talk about.
                               ''')

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
    "system_message": system_message
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [9]:
def google_search(query: str):
    """A google search tool"""
    return f"Here is a google search result for {query}"

def save_interview(interview_text: str):
    """A tool to save an interview"""
    return f"Here is the interview saved: {interview_text}"

def read_pdf_from_url(url: str):
    """A tool to read a pdf from a url"""
    return f"Here is the pdf from the url: {url}"

def get_pokemon(pokemon_name: str):
    """A tool to get a pokemon"""
    return f"Here is the pokemon: {pokemon_name}"

google_search = StructuredTool.from_function(google_search)
save_interview = StructuredTool.from_function(save_interview)
read_pdf_from_url = StructuredTool.from_function(read_pdf_from_url)
pokemon = StructuredTool.from_function(get_pokemon)

In [10]:
google_search .args_schema.schema_json()

'{"title": "google_searchSchemaSchema", "type": "object", "properties": {"query": {"title": "Query", "type": "string"}}, "required": ["query"]}'

In [11]:
google_search .run("Data")

[tool/start] [1:tool:google_search] Entering Tool run with input:
"Data"
[tool/end] [1:tool:google_search] s] Exiting Tool run with output:
"Here is a google search result for Data"


'Here is a google search result for Data'

In [13]:
llm = ChatOpenAI(temperature=0)

tools = [
    google_search,
    save_interview,
    read_pdf_from_url,
    pokemon,
]
agent = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True,
    memory=memory,
    agent_kwargs=agent_kwargs,
)

In [14]:
agent.run("My name is James, I'm a computer programmer and I like to play cosy games.")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "My name is James, I'm a computer programmer and I like to play cosy games.",
  "memory": []
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Act as a helpful AI interview assistant, \n                               You will either be having a conversation to generate content or calling a function to help the user.\n                               \n                               You must follow the following principles:\n                               - Always be asking the user a follow up question, especially after returning the results from a previous tool.\n                               - Ask one question at a time.\n                               - You must always be in interview mode (asking questions) or in tool mode (calling a function). \n                               - Ask the user what topic they want to talk about.\n         

"Nice to meet you, James! As a computer programmer, I'm sure you have a lot of interesting experiences and insights. And playing cosy games sounds like a great way to relax. Is there anything specific you would like to talk about or any questions you have?"

In [15]:
agent.run('What is my name and my profession?')

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Act as a helpful AI interview assistant, \n                               You will either be having a conversation to generate content or calling a function to help the user.\n                               \n                               You must follow the following principles:\n                               - Always be asking the user a follow up question, especially after returning the results from a previous tool.\n                               - Ask one question at a time.\n                               - You must always be in interview mode (asking questions) or in tool mode (calling a function). \n                               - Ask the user what topic they want to talk about.\n                               \nHuman: My name is James, I'm a computer programmer and I like to play cosy 

'Your name is James and you mentioned that you are a computer programmer. Is there anything specific you would like to discuss related to programming or cosy games?'